In [1]:
!pip install pyspark;


In [1]:
import pyspark

# Know your machine resources

In [2]:
# CPU Cores
!sysctl -n hw.logicalcpu

# Memory in Bytes
!sysctl -n hw.memsize

8
17179869184


In [3]:
# Memory in GB
17179869184/(1024**3)

16.0

In [4]:
MAX_MEMORY = '4g'
MAX_MEMORY_OVERHEAD = '512m'
MAX_DRIVER_MEMORY = '1g'

In [12]:
conf = pyspark.SparkConf().setMaster("local[2]") \
        .set('spark.executor.memory', MAX_MEMORY) \
        .set('spark.executor.memoryOverhead', MAX_MEMORY_OVERHEAD) \
        .set('spark.driver.memory', MAX_DRIVER_MEMORY)
        # .set('spark.executor.heartbeatInterval', 10000) \
        # .set('spark.network.timeout', 10000) \
        # .set('spark.core.connection.ack.wait.timeout', '3600') \

In [13]:
from pyspark.sql import SparkSession

In [14]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("StarWars - Data Quality Project") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

In [15]:
spark = init_spark()

In [16]:
df = spark.read.csv('star_wars_reviews.csv',header=True)

In [17]:
df.show(5)

+---------+----------------+--------------+--------------------+--------------------+-----------------+----------+------------+
|review_id|       fav_heroe|   fav_villain|            fav_film|      fav_soundtrack|    fav_spaceship|fav_planet|   fav_robot|
+---------+----------------+--------------+--------------------+--------------------+-----------------+----------+------------+
|        0|Anakin Skywalker|    Darth Maul|Episode IV - A Ne...|   Accross the Stars|Naboo Starfighter|  Tatooine|       R2-D2|
|        1|Anakin Skywalker|    Darth Maul|Episode IV - A Ne...|     The Throne Room|Naboo Starfighter|  Tatooine|Battle Droid|
|        2|  Luke Skywalker|   Count Dooku|Episode V - The E...|Star Wars (Main T...|Millennium Falcon|     Endor|       R2-D2|
|        3|Anakin Skywalker|Wilhuff Tarkin|Episode VI - Retu...|     The Throne Room|Millennium Falcon|  Tatooine|       R2-D2|
|        4|            Yoda|     Palpatine|Episode IV - A Ne...|Star Wars (Main T...|Millennium Falcon| 